In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import emoji
import spacy
import re
from autocorrect import Speller
import enchant
from enchant.checker import SpellChecker

In [2]:
df = pd.read_csv("./data/output.csv")

In [3]:
# "review_text" nan are to be deleted. Other can stay.
df = df.dropna(subset=["review_text"])

# Remove invalid entries
df = df.loc[df["entry_date"] != "entry_date"]

# Remove duplicate entries
df = df.drop_duplicates(["offer_ref", "entry_id", "review_text"]).drop_duplicates("review_text")

# fix data types
df["entry_date"] = pd.to_datetime(df["entry_date"])
df["purchase_date"] = pd.to_datetime(df["purchase_date"])
df["entry_id"] = df["entry_id"].astype(int)
df["offer_ref"] = df["offer_ref"].astype(int)
df["score"] = df["score"].astype(float)

# Get Sentiment Cases based on score
df["sentiment"] = df["score"].apply(lambda x: "Positive" if x >= 4 else "Negative" if x <= 2 else "Neutral")

In [4]:
to_clean = df[["review_text", "sentiment"]].copy()

In [5]:
# Get rid of newlines
to_clean["review_text"] = to_clean["review_text"].replace("[\t\r\n\v\f\ufeff]", " ", regex=True)

In [6]:
# Remove whitespaces
to_clean["review_text"] = to_clean["review_text"].replace(" +", " ", regex=True)
to_clean[to_clean["review_text"].str.contains("  ")]

,review_text,sentiment


In [7]:
to_clean["has_emoji"] = to_clean["review_text"].transform(lambda x: np.any([emoji.is_emoji(c) for c in x]))
to_clean["all_emoji"] = to_clean["review_text"].transform(lambda x: np.all([emoji.is_emoji(c) for c in x]))

In [8]:
# 460 Reviews containing emojis
print(len(to_clean[to_clean["has_emoji"]]))
# 42 reviews containing only emojis
print(len(to_clean[to_clean["all_emoji"]]))

460
42


In [9]:
# You can sort of guess the sentiment of the review based on the emojis. But it's not consistent.
to_clean[to_clean["all_emoji"]].head(10)

,review_text,sentiment,has_emoji,all_emoji
0,😑😑,Negative,True,True
1091,😐,Negative,True,True
9417,👍,Neutral,True,True
10049,👍👍👍👍👍,Positive,True,True
10954,🤗,Positive,True,True
11175,🥳🎉,Positive,True,True
13724,👍🏻👍🏻,Negative,True,True
14707,👍👍,Neutral,True,True
24200,🙂,Positive,True,True
25320,😫,Negative,True,True


In [10]:
# 1927 reviews containing special unicode characters
to_clean[to_clean["review_text"].str.contains("[\t\r\n\v\f\ufeff]")]

,review_text,sentiment,has_emoji,all_emoji


In [11]:
to_clean["has_ok"] = to_clean["review_text"].str.lower().str.contains("\Wok\W")
to_clean[to_clean["has_ok"]] = to_clean[to_clean["has_ok"]].replace(r"\bok\b", "Ok", regex=True)

In [12]:
to_clean[to_clean["has_ok"] & (to_clean["sentiment"] != "Positive")]["review_text"].iloc[2]

'Kupiłem za namową sąsiadów, którzy zachwycali się możliwością pieczenia 2 potraw jednocześnie. Dual Cooki nie są drogie, miałem Ok. 2000 zł na piekarnik (poprzedni zepsuł się po 13 latach, ale wiem, że takich urządzeń już nie ma...). Zdecydowałem się na ten model choć już w sklepie pojawiły się pierwsze wątpliwości... drzwiczki piekarnika trzaskają jak w maluchu, obudowa drzwi jest w pełni plastikowa i nie znalazłem żadnego uszczelnienie tej przegrody rozdzielającej piekarnik na 2 części. Sprzedawca stwierdził, że są to bzdety nie mające wpływu na jakość pieczenia i szczerze mówiąc przekonał mnie. Piekę głównie mięsiwa, a żona ciasta - piekarnik działa praktycznie codziennie, używamy go także do podgrzewania (nie lubię mikrofali). Rozczarowanie przyszło już po pierwszym pieczeniu - piekarnik bardzo długo się nagrzewa - temperaturę 220 stopni osiągnął po 20 minutach, mimo, że termostat pokazywał taką temperaturę już po 8-10 minutach - własny termometr pokazał, że piekarnik przekłamuje.

In [13]:
# fix_ok_in_string assumes that ok. (circa) happens once per review. We can safely assume that.
to_clean[to_clean["review_text"].transform(lambda x: len(re.findall(r"\bok\b.? [0-9]+", x))) > 1]

,review_text,sentiment,has_emoji,all_emoji,has_ok


In [14]:
# Function discriminates between ok meaning good and ok meaning circa 
# which is a short form of około in polish language.
# Ok has it's own fix because it is very common in reviews and gives positive sentiment.
def fix_ok_in_string(text):
    circa_match = re.search(r"\bok\b.? [0-9]+", text.lower())
    if circa_match is not None:
        circa_match = circa_match.span()
        return re.sub(r"\b(ok|Ok|OK|oK)\b", "Ok", text[:circa_match[0]]) + text[circa_match[0]:circa_match[1]] + re.sub(r"\b(ok|Ok|OK|oK)\b", "Ok", text[circa_match[1]:])
    else:
        return re.sub(r"\b(ok|Ok|OK|oK)\b", "Ok", text)

In [15]:
fix_ok_in_string("ok. lalala ok. 100kg ok!")

'Ok. lalala ok. 100kg Ok!'

In [16]:
to_clean["review_text"] = to_clean["review_text"].transform(fix_ok_in_string)

In [17]:
to_clean["review_text"] = to_clean["review_text"].transform(lambda x: re.sub("[0-9]+", "", x))

In [18]:
nlp = spacy.load("pl_core_news_lg")

In [19]:
spell = Speller("pl", only_replacements=True)

In [20]:
chkr = SpellChecker("pl_Pl")

In [21]:
d_typo = enchant.Dict("pl_PL")

In [22]:
checker_test = to_clean["review_text"].iloc[:2000].transform(lambda x: nlp(x))

In [23]:
# 42% of first 2000 reviews contain typos
# pyenchany seems good at detecting typos however it seems bad at fixing typos So we'll have to be careful with it.
checker_test[checker_test.transform(lambda x: all([d_typo.check(token.text.lower()) for token in x if not token.is_punct])) == False]

0                                                  (😑, 😑)
1       (Lekkie, i, jak, jesteś, na, kuligu, to, szypk...
9       (Niepełny, produkt, ,, wprowadzanie, w, błąd, ...
13      (Bosch, to, już, nie, ta, sama, marka, ,, co, ...
15      (PIEKARNIK, BARDZO, DODRZE, FUNKCJONUJE, ,, CI...
                              ...                        
3635    (nie, polecam, ,, po,  , latach, użytkowania, ...
3638    (Wyglada, Ok, i, tyle, ., Regulacja, temperatu...
3650    (Urządzenie, mam, ponad, rok, ., Używane, głów...
3658    (Jestem, z, tego, robota, zadowolona, ,, ale, ...
3659    (Bateria, bardzo, praktyczna, w, użytkowaniu, ...
Name: review_text, Length: 818, dtype: object

In [43]:
[d_typo.suggest(token.text)[0] if (not d_typo.check(token.text)) and (not token.is_punct) else token for token in checker_test.iloc[1]]

[Lekkie,
 i,
 jak,
 jesteś,
 na,
 kuligu,
 to,
 'szypo',
 spadasz,
 'słabe',
 sanki,
 .,
 NIE,
 POLECAM,
 !,
 !,
 !,
 !,
 !,
 !,
 !,
 !]

In [47]:
to_clean["review_text"] = to_clean["review_text"].transform(nlp)

In [48]:
# Taker 44 mins.
# to_clean["review_text"].transform(lambda x: [d_typo.suggest(token.text)[0] if (not token.is_punct) and (not d_typo.check(token.text)) and (chkr.suggest(token.text))  else token for token in x]).to_csv("Testing Typo Checking.csv")
# to_clean["review_text"].to_csv("Comparison to a Typo fix.csv")

In [50]:
to_clean["sentiment"].to_csv("sentiment provision.csv")

In [45]:
result = checker_test.transform(lambda x: [d_typo.suggest(token.text)[0] if (not token.is_punct) and (not d_typo.check(token.text)) and (chkr.suggest(token.text))  else token for token in x])